# Tutorial for demonstrating Super Kriging

This file is meant to demonstrate the use of gstlearn for Super Kriging. It is run on several 2-D cases.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
return false;
}

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt

Setting some global variables

In [ ]:
# Set the Global Options
verbose = True
flagGraphic = True

# Define the Space Dimension
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim)

# Set the Seed for the Random Number generator
gl.law_set_random_seed(5584)

# Next option performs a verbose kriging of one target node (lots of outputs)
flagDebug = True

We define the grid on which all calculations will be performed. The grid is constituted of square meshes and has a rectangular extension of 200 by 150.

In [ ]:
grid = gl.DbGrid.create(nx=[200,150],dx=[1.,1.])
grid

# Poisson Data Set

In this paragraph, we generate a Poisson Data Set (data uniformy distributed along each space dimension) which covers the Grid expansion area. The Data Set contains 100 samples.

In [ ]:
coormin = grid.getCoorMinimum()
coormax = grid.getCoorMaximum()
nech = 100
data = gl.Db.createFromBox(nech, coormin, coormax)

A variable is generated on this data set, as the result of a non-conditional simulation with a build-in model.

In [ ]:
model = gl.Model()
model.addCovFromParam(gl.ECov.SPHERICAL, range=40, sill=0.7)
model.addCovFromParam(gl.ECov.NUGGET, sill = 0.3)
err = gl.simtub(None, dbout=data, model=model)
data

In [ ]:
ax = gp.point(data,color_name="Simu",figsize=[7,7], aspect=1)

## Defining standard neighborhood

We first define a standard Moving Neighborhood.

In [ ]:
nmini = 1
nmaxi = 10
radius = 30.
nsect = 8
nsmax = 3
neigh = gl.NeighMoving.create(flag_xvalid=False, nmaxi=nmaxi, radius=radius, nmini=nmini, 
                              nsect=nsect, nsmax=nsmax)
neigh

Checking the neighborhood around a central grid node

In [ ]:
node = 15300
target = grid.getSampleCoordinates(node)
ranks = gl.neigh_calc(data, grid, neigh, node)

dataSel = data.clone()
dum = dataSel.addSelectionByRanks(ranks)

In [ ]:
ax = gp.point(data, figsize=[7,7], aspect=1)
ax = gp.point(dataSel, ax=ax, color='blue')
ax = gp.sample(target, ax=ax, color='black')
ax = gp.curve(grid.getCellEdges(node), ax=ax)
ax = gp.curve(neigh.getEllipsoid(target), ax=ax)
ax = gp.multisegments(target,neigh.getSectors(target), ax=ax)

### Point Kriging

Performing a Point Kriging with the current Neighborhood feature

In [ ]:
err = gl.kriging(data, grid, model, neigh, 
                 namconv=gl.NamingConvention("Point_Kriging"))

In [ ]:
if flagDebug:
    err = gl.krigtest(data, grid, model, neigh, node)

The Point Kriging results are displayed (overlaying the control data points)

In [ ]:
ax = gp.grid(grid, name="Point_Kriging*estim", 
             figsize=[7,7], aspect=1,
             title="Point Kriging with Standard Neighborhood")
ax = gp.point(data, color="white", ax=ax)

We also display the standard deviation map of the Estimation error

In [ ]:
ax = gp.grid(grid, name="Point_Kriging*stdev", 
             figsize=[7,7], aspect=1,
             title="Error for Point Kriging with Standard Neighborhood")
ax = gp.point(data, color="white", ax=ax)

### Block Kriging

Performing a Block Kriging with the current Neighborhood feature. Note that the discretization parameters have been set to small numbers in order to let the calculations be performed in a reasonable time frame (for a demonstration file).

In [ ]:
err = gl.kriging(data, grid, model, neigh, calcul=gl.EKrigOpt.BLOCK, ndisc=[5,5],
                namconv=gl.NamingConvention("Block_Kriging"))

In [ ]:
if flagDebug:
    err = gl.krigtest(data, grid, model, neigh, node, calcul=gl.EKrigOpt.BLOCK, ndisc=[5,5])

The Block Kriging results are displayed (overlaying the control data points)

In [ ]:
ax = gp.grid(grid, name="Block_Kriging*estim", 
             figsize=[7,7], aspect=1,
             title="Block Kriging with Standard Neighborhood")
ax = gp.point(data, color="white", ax=ax)

We also display the standard deviation map of the Estimation error

In [ ]:
ax = gp.grid(grid, name="Block_Kriging*stdev", 
             figsize=[7,7], aspect=1,
             title="Error for Block Kriging with Standard Neighborhood")
ax = gp.point(data, color="white", ax=ax)

Comparing the Estimation maps

In [ ]:
ax = gp.correlation(grid,"Point_Kriging*estim","Block_Kriging*estim")

Comparing the Error Estimation maps

In [ ]:
ax = gp.correlation(grid,"Point_Kriging*stdev","Block_Kriging*stdev")

The difference is not very impressive due to the small size of block extensions

## Defining variable Block Extensions

In this section, we will generate variables in the Grid File, which contain the cell extension. The square Block size is fixed to 50.

In [ ]:
size = 50.
iuid = grid.addColumnsByConstant(1, size, "X-ext", gl.ELoc.BLEX, 0)
iuid = grid.addColumnsByConstant(1, size, "Y-ext", gl.ELoc.BLEX, 1)

The now check the neighborhood feature which consists in forcing any sample located within the cell extension centered on the target grid node.

In [ ]:
neigh.setForceWithinBlock(True)
neigh.display()

We check the new neighborhood on the same target grid node as before

In [ ]:
ranks = gl.neigh_calc(data, grid, neigh, node)

dataSel = data.clone()
dum = dataSel.addSelectionByRanks(ranks)

The next figure displays the samples selected in the neighborhood of the target node (same as before). As expected all samples lying within the super_block centered on the target node are considered (i.e. 34) rather than the samples which would have been considered in the standard neighborhood case (i.e. 15).

In [ ]:
ax = gp.point(data, figsize=[7,7], aspect=1)
ax = gp.point(dataSel, ax=ax, color='blue')
ax = gp.sample(target, ax=ax, color='black')
ax = gp.curve(grid.getCellEdges(node), color='black', ax=ax)

We now perform the Super Kriging which is nothing but a standard Kriging with the new neighborhood feature (demonstrated above).

In [ ]:
err = gl.kriging(data, grid, model, neigh, calcul=gl.EKrigOpt.BLOCK, ndisc=[5,5],
                 namconv=gl.NamingConvention("Super_Kriging"))

In [ ]:
if flagDebug:
    err = gl.krigtest(data, grid, model, neigh, node, calcul=gl.EKrigOpt.BLOCK, ndisc=[5,5])

The results of the Super Kriging are visulized in the next figure, together with the ones of the standard neighborhood.

In [ ]:
fig = plt.figure(figsize=(20,10))

ax1 = fig.add_subplot(1,2,1)
ax1 = gp.grid(grid, name="Super_Kriging*estim", 
              figsize=[7,7], aspect=1,
              title="Super Block Kriging", ax=ax1)
ax1 = gp.point(data, color="white", ax=ax1)

ax2 = fig.add_subplot(1,2,2)
ax2 = gp.grid(grid, name="Block_Kriging*estim", 
              figsize=[7,7], aspect=1,
              title="Block Kriging with Standard Neighborhood", ax=ax2)
ax2 = gp.point(data, color="white", ax=ax2)

We also display the standard deviation map of the Super Kriging Estimation error

In [ ]:
fig = plt.figure(figsize=(20,10))

ax1 = fig.add_subplot(1,2,1)
ax1 = gp.grid(grid, name="Super_Kriging*stdev", 
              figsize=[7,7], aspect=1,
              title="Error for Super Block Kriging", ax=ax1)
ax1 = gp.point(data, color="white", ax=ax1)

ax2 = fig.add_subplot(1,2,2)
ax2 = gp.grid(grid, name="Block_Kriging*stdev", 
              figsize=[7,7], aspect=1,
              title="Error for Block Kriging with Standard Neighborhood", ax=ax2)
ax2 = gp.point(data, color="white", ax=ax2)

Comparing with Block Kriging with standard block extension (equal to the grid mesh)

In [ ]:
ax = gp.correlation(grid,"Block_Kriging*estim","Super_Kriging*estim")

Comparing the error maps

In [ ]:
ax = gp.correlation(grid,"Block_Kriging*stdev","Super_Kriging*stdev")